# $\verb|MakeMyTrip.com|$ 

Make sure to cross check XPath for each address.

In [1]:
from selenium import webdriver
from parsel import Selector
import os, time
import pandas as pd
import re
import numpy as np

//Deprecated

Change the BASE_URL with the string format:{} as the url also contains visitorID, which might track the frequency of Queries

UPDATE: You can change the initial numbers of visitorID to any number, so suggest doing that too. 

//

Login with your 2nd account first.

In [2]:
CITY = 'Raipur'#Enter a city
BASE_URL = 'https://www.makemytrip.com/hotels/'

After each city change in BASE_URL, please check if the current webpage's city is same as the previous, or print it out. As when there's no hotels for that search it loads the same back.

In [3]:
SEARCH_AREA_XPATH = '/html/body/div/div/div[2]/div/div/div[2]/div/div[1]/div[1]/label'
ACTUAL_SEARCH_AREA_XPATH = '/html/body/div/div/div[2]/div/div/div[2]/div/div[1]/div[1]/div[1]/div/div/div/input'
FIRST_OR_SEC_SUGG_XPATH = '/html/body/div/div/div[2]/div/div/div[2]/div/div[1]/div[1]/div[1]/div/div/div/div/div/ul/li[{}]/div/div' #use .format(1) or .format(2)

#ROOMS_GUESTS_XPATH = '/html/body/div/div/div[2]/div/div/div[2]/div/div/div[4]/label'
#ROOMS_GUESTS_APPLY_XPATH = '/html/body/div/div/div[2]/div/div/div[2]/div/div/div[4]/div[1]/div[2]/button[2]'

SEARCH_XPATH = '/html/body/div/div/div[2]/div/div/div[2]/div/p/button'
               
CLOSE_BUTTON_XPATH = '/html/body/div[1]/div/div[2]/div[3]/span'

SEARCH_HOTEL_NAME_XPATH = '/html/body/div[1]/div/div[2]/div[2]/div[1]/div/div/div[2]/form/div/div/input'
                          
#Enter the keyword 'oyo'

OYO_RESULTS_LINK_XPATH = '/html/body/div[1]/div/div[2]/div[2]/div[1]/div/div/div[2]/form/div/div/div/ul/li[{}]/a/attribute::href'


Extract Links first

In [64]:
driver = webdriver.Chrome(executable_path='/home/animesh/Meltwater Winter 2019/Web Scraping Tasks/OYO Rooms Reviews/Booking.com/chromedriver')

In [65]:
driver.get(BASE_URL)

### Testing a sample use.

1. Entering City to Search

In [27]:
search_area = driver.find_element_by_xpath(SEARCH_AREA_XPATH)
search_area.click()
search_area.send_keys(CITY)

2. CLicking first(or second) suggestion 

In [28]:
try:
    suggestion = driver.find_element_by_xpath(FIRST_OR_SEC_SUGG_XPATH.format(1))
except:
    suggestion = driver.find_element_by_xpath(FIRST_OR_SEC_SUGG_XPATH.format(2))
    
suggestion.click()

3. Clicking Search Button

In [29]:
driver.find_element_by_xpath(SEARCH_XPATH).click()

4. Look for the "CLOSE" Button, and if it's there, then click it.

In [30]:
try:
    driver.find_element_by_xpath(CLOSE_BUTTON_XPATH).click()
except:
    print("No Close button found, continuing....")

5. Search for the Keyword 'OyO' in Hotel Name Search Area

In [31]:
hotel_name = driver.find_element_by_xpath(SEARCH_HOTEL_NAME_XPATH)
hotel_name.send_keys('OyO')

6. Extract all the links

In [32]:
i = 0
urls = []
while True:
    i+=1
    try:
        sel = Selector(text = driver.page_source)
        link = 'https://www.makemytrip.com' + sel.xpath(OYO_RESULTS_LINK_XPATH.format(i)).extract()[0]
        urls.append(link)
    except:
        print("Scraped {} hotels... Breaking off.".format(len(urls)))
        break

Scraped 10 hotels... Breaking off.


So now, we have links of all the OyO hotels in the city.

Defining, above stuff as a function below:-

In [6]:
def Scrape_hotels(driver,CITY, return_csv = False):
    
    '''driver should be at BASE_URL'''
    
    driver.find_element_by_xpath(SEARCH_AREA_XPATH).click()
    #search_area = driver.find_element_by_xpath(ACTUAL_SEARCH_AREA_XPATH)
    time.sleep(2)
    driver.find_element_by_xpath(ACTUAL_SEARCH_AREA_XPATH).send_keys(CITY)


    time.sleep(2)


    
    
    try:
        suggestion = driver.find_element_by_xpath(FIRST_OR_SEC_SUGG_XPATH.format(1))
    except:
        suggestion = driver.find_element_by_xpath(FIRST_OR_SEC_SUGG_XPATH.format(2))

    suggestion.click()


    driver.find_element_by_xpath(SEARCH_AREA_XPATH).click()
    time.sleep(1)
    driver.find_element_by_xpath(ACTUAL_SEARCH_AREA_XPATH).send_keys(CITY)

    try:
        suggestion = driver.find_element_by_xpath(FIRST_OR_SEC_SUGG_XPATH.format(1))
    except:
        suggestion = driver.find_element_by_xpath(FIRST_OR_SEC_SUGG_XPATH.format(2))

    suggestion.click()


    
    
    driver.find_element_by_xpath(SEARCH_XPATH).click()


    time.sleep(0.5)


    try:
        driver.find_element_by_xpath(CLOSE_BUTTON_XPATH).click()
    except:
        print("No Close button found, continuing....")


    hotel_name = driver.find_element_by_xpath(SEARCH_HOTEL_NAME_XPATH)
    hotel_name.send_keys('OyO')


    i = 0
    urls = []
    time.sleep(1)
    sel = Selector(text = driver.page_source)
    while True:
        i+=1
        try:
            link = 'https://www.makemytrip.com' + sel.xpath(OYO_RESULTS_LINK_XPATH.format(i)).extract()[0]
            urls.append(link)
        except:
            print("Scraped {} hotels... Breaking off.".format(len(urls)))
            break

    if return_csv == False:
        return pd.DataFrame(urls,columns=['Links'])
    
    return pd.DataFrame(urls,columns=['Links']).to_csv('{}_MMT_OYO_Hotels.csv'.format(CITY))

Testing above function

In [36]:
driver.get(BASE_URL)

In [37]:
Scrape_hotels(driver,'Ghaziabad',return_csv = True)

No Close button found, continuing....
Scraped 10 hotels... Breaking off.


## Using it to get OYO hotel links from top cities by Tourism.

In [7]:
top_cities = ['Delhi','Mumbai','Chennai','Agra','Jaipur','Kolkata']

In [8]:
links = []
for city in top_cities:
    driver.get(BASE_URL)
    links.append(Scrape_hotels(driver,city,return_csv = False))

No Close button found, continuing....
Scraped 10 hotels... Breaking off.


KeyboardInterrupt: 

In [107]:
df = pd.DataFrame()

for i in range(len(top_cities)):
    df = df.append(links[i])

In [108]:
df

,Links
0,https://www.makemytrip.com/hotels/hotel-detail...
1,https://www.makemytrip.com/hotels/hotel-detail...
2,https://www.makemytrip.com/hotels/hotel-detail...
3,https://www.makemytrip.com/hotels/hotel-detail...
4,https://www.makemytrip.com/hotels/hotel-detail...
...,...
5,https://www.makemytrip.com/hotels/hotel-detail...
6,https://www.makemytrip.com/hotels/hotel-detail...
7,https://www.makemytrip.com/hotels/hotel-detail...
8,https://www.makemytrip.com/hotels/hotel-detail...


In [109]:
df.to_csv('MMT_OYO_Top_Cities_Hotels.csv')

## Scraping the Reviews Now.

Each Page contains 20 reviews at max.

In [12]:
df = pd.read_csv('MMT_OYO_Top_Cities_Hotels.csv')

In [17]:
driver.get(df.Links.iloc[7])

### Final Ratings

In [18]:
FINAL_RATING_XPATH = '/html/body/div[1]/div/div[2]/section[5]/div[2]/div[1]/div[1]/div[1]/div/span'

In [19]:
driver.find_element_by_xpath(FINAL_RATING_XPATH).text

'4'

### Total Reviews

In [20]:
TOTAL_REVIEWS_XPATH = '/html/body/div[1]/div/div[2]/section[5]/div[2]/div[1]/div[1]/div[1]/div/div/p[2]'

In [21]:
int(driver.find_element_by_xpath(TOTAL_REVIEWS_XPATH).text[:-7])

27

### Full Reviews

In [22]:
REVIEW_RATING_XPATH = '/html/body/div[1]/div/div[2]/section[5]/div[2]/div[{}]'

In [23]:
driver.find_element_by_xpath(REVIEW_RATING_XPATH.format(1)).text

'4\n51 RATINGS\n27 REVIEWS\n2.0\nSecurity\n4.1\nRoom\n4.0\nHospitality\nAll Reviews\nRoom Quality\nRoom Amenities\nBathroom Hygiene\nStaff Courtesy\nService Quality\nCheck-in\nRoom Cleanliness\nBathroom Cleanliness\nValue for Money'

In [24]:
driver.find_element_by_xpath(REVIEW_RATING_XPATH.format(2)).text

'Good\nModi Jayendra, Dec 15, 2019\nGood room, good service, good location\n2.0'

.format(1) on REVIEW_RATING_XPATH gives the Review Header
.format(2n) gives individual review with full details as text

## Next Page Button

We're using CSS Selector as XPath changes for it (and a lot other parameters too)

In [25]:
NEXT_PAGE_BUTTON = '#detpg_review_ratings_pagination > ul > li:nth-child(8) > a' 
NEXT_PAGE_BUTTON_aliter_1 = '#detpg_review_ratings_pagination > ul > li:nth-child(6) > a'
                            

In [ ]:
driver.find_element_by_css_selector(NEXT_PAGE_BUTTON).click()

Now repeat the above process of Extracting Full Reviews again on each review page

### Total Pages = ceil(Total_Reviews / 20)

In [27]:
import numpy as np

In [48]:
np.ceil(138/20)

7.0

In [158]:
total_reviews = 8
review_on_first_page = total_reviews if total_reviews<20 else 20

In [159]:
review_on_first_page

8

In [28]:
driver.find_element_by_css_selector('#UserReviews > div._RatingReview > div:nth-child(4)').text

"Nightmare at Oyo\nAnil Kumar, Dec 11, 2019\nIt's my worst experience stay at this hotel, I would never recommend to anyone to stay at this hotel. No clean room, no water, it's really very terrible\n1.0"

## Testing out and making the selectors on given review page

In [29]:
Review_title_selector = '#UserReviews > div._RatingReview > div.reviewBox > div > div > p.latoBold.font18.lineHight22.greenText'
Name_Date_selector = '#UserReviews > div._RatingReview > div.reviewBox > div > div > p.grayText.appendBottom10'
Review_selector = '#UserReviews > div._RatingReview > div.reviewBox > div > div > p.font14.lineHight22'
Review_rating = '#UserReviews > div._RatingReview > div.reviewBox > div > span'

In [208]:
sel = Selector(text = driver.page_source)

In [254]:
Ratings = sel.css(Review_rating).extract()
for i,r in enumerate(Ratings):
    Ratings[i] = float(r[-10:-7])

In [256]:
Ratings

[2.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 5.0,
 2.0,
 5.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 2.0,
 3.0,
 3.0]

In [209]:
Review_titles = sel.css(Review_title_selector).extract()
for i,title in enumerate(Review_titles):
    Review_titles[i] = title.replace('<p class="latoBold font18 lineHight22 greenText">','').replace('</p>','')

In [210]:
Review_titles

['Nice location but pathetic room and food',
 'Hotel regalia',
 "It's really a good hotel",
 'Excellent Stay',
 'Everything is nice',
 'Very good accommodation',
 'Location',
 'Excellent Stay',
 'Poor Stay',
 'Very peaceful location',
 'Average',
 'Nice Neat &amp; Clean',
 'Total value for money,home away Home Feel.',
 'GOOD PLACE IF YOU HAVE WORK IN NEHRU PLACE',
 'average 2 star facilities',
 'Very friendly and helpful staff - from the porters to the management...',
 'Was as expected',
 'We would NOT recommend to anyone other than "backpackers!"',
 'Can improve cleanliness',
 'Average Stay']

In [211]:
Reviews = sel.css(Review_selector).extract()
for i,review in enumerate(Reviews):
    Reviews[i] = review.replace('<p class="font14 lineHight22">','').replace('</p>','')

In [212]:
Reviews

["I stayed there for a day and at night I couldn't sleep properly due to mosquitoes and bed bugs. I have complained about this to the hotel staff but their responses aren't satisfactory.",
 'The hotel is in good. So the location is good but facilities in the hotel are not up to the mark. There was no water in the morning which is absolutely unacceptable.',
 'I am very happy to stay there because all staff are very polite and always ready for serve you. Average stay there.',
 'Nice stay, nice service and best location. Overall nice experience.',
 'Awesome.',
 'Previously also I stayed there and provided good service. Good location. Good staff. And really good food. Am happy.',
 "Yesterday night I placed a order in zomato, but the was unable to find the place and I didn't provide him landmarks because the landmarks is far away from hotel.<br>I am working guy so I spend only in night, but the thing which I don't like is breakfast timing. I said to them that I have to leave early so provid

In [213]:
Names_dates = sel.css(Name_Date_selector).extract()

In [214]:
for i,nd in enumerate(Names_dates):
    Names_dates[i] = nd.replace('<p class="grayText appendBottom10">','').replace('</p>','')

In [215]:
Names_dates

['Shashank Sinha, Sep 17, 2018',
 'Disha Khandelwal, Sep 04, 2018',
 'Amrit Kumar, Aug 22, 2018',
 'varun nayak, Aug 17, 2018',
 'Annee Negi, Aug 05, 2018',
 'Ajit Menon, Aug 01, 2018',
 'Ahsan Ahmed, Jul 24, 2018',
 'abhijeet sharma, Jul 18, 2018',
 'sumeer Sharma, Jul 16, 2018',
 'bhawna singh, Jul 02, 2018',
 'MIHIR, Apr 22, 2018',
 'BALRAJ SIDHU, Jun 17, 2016',
 'Meenakshi Pai, Jun 17, 2016',
 'Deepak Goyal, Jun 17, 2016',
 'Prakash Panachikkuth, Jun 17, 2016',
 'Matthew SinghToor, Jun 17, 2016',
 'Panachikkuth Prakash, Jun 17, 2016',
 'M ichael Bleicher, Jun 17, 2016',
 'Anubhav Khandelwal, Jun 17, 2016',
 'Hemachandra Singh, Dec 19, 2012']

In [290]:
driver = webdriver.Chrome('/home/animesh/Meltwater Winter 2019/Web Scraping Tasks/OYO Rooms Reviews/Booking.com/chromedriver')

In [53]:
driver.get('https://www.makemytrip.com/hotels/hotel-details/?hotelId=201812051040091117&checkin=12172019&checkout=12182019&city=CTDEL&country=IN&lat=28.500557&lng=77.185776&locusId=CTDEL&locusType=city&roomStayQualifier=2e0e&searchText=Delhi%2C%20India&seekSummaryRequired=true&type=hotel&visitorId=567da42f-26c6-4580-ae9c-e6bfa8f6eadd')

In [54]:
driver.find_element_by_css_selector("#UserReviews > div._RatingReview > div.reviewHeader > div.headerLeft > div.userRating.yellowBg.yellowBorder > div > div > p:nth-child(2)").text

'52 REVIEWS'

In [36]:
def Review_Extractor(driver):
    '''
    driver must have loaded the hotel link and it should be on page 1 of the revies page
    '''
    
    time.sleep(1)
    
    try:
        total_reviews = int(driver.find_element_by_css_selector("#UserReviews > div._RatingReview > div.reviewHeader > div.headerLeft > div.userRating.yellowBg.yellowBorder > div > div > p:nth-child(2)").text[:-7])
    except:                                                     #"#UserReviews > div._RatingReview > div.reviewHeader > div.headerLeft > div.userRating.yellowBg.yellowBorder > div > div > p:nth-child(2)"
        try:
            total_reviews = int(driver.find_element_by_css_selector("#UserReviews > div._RatingReview > div.reviewHeader > div.headerLeft > div.userRating.lightGreenBg.lightGreenBorder > div > div > p:nth-child(2)").text[:-7])
        except:                                                     
            total_r
    total_pages = np.ceil(total_reviews/20)
    
    print("Total Reviews on this Link: {}".format(total_reviews))
    
    final_matrix = []
    
    for j in range(int(total_pages)):

        sel = Selector(text = driver.page_source)
        
        Ratings = sel.css(Review_rating).extract()
        for i,r in enumerate(Ratings):
            Ratings[i] = float(r[-10:-7])

        Review_titles = sel.css(Review_title_selector).extract()
        for i,title in enumerate(Review_titles):
            Review_titles[i] = title.replace('<p class="latoBold font18 lineHight22 greenText">','').replace('</p>','')

        Reviews = sel.css(Review_selector).extract()
        for i,review in enumerate(Reviews):
            Reviews[i] = review.replace('<p class="font14 lineHight22">','').replace('</p>','')

        Names_dates = sel.css(Name_Date_selector).extract()
        for i,nd in enumerate(Names_dates):
            Names_dates[i] = nd.replace('<p class="grayText appendBottom10">','').replace('</p>','')
            
        try:
            total_matrix = np.c_[Review_titles,Names_dates,Ratings,Reviews]
        except Exception as e:
            till = min(len(Review_titles),len(Names_dates),len(Ratings),len(Reviews))
            total_matrix = np.c_[Review_titles[:till],Names_dates[:till],Ratings[:till],Reviews[:till]]
            print("Continuing....")
            continue
                
                
        final_matrix.append(total_matrix)
        
        print("Amount of Reviews' Pages Scraped: {}".format(j+1))
        print("Amount of Reviews Scraped on this Review Page: {}".format(len(Reviews)))
        
        try:
            driver.find_element_by_css_selector(NEXT_PAGE_BUTTON).click()
        except:
            try:
                driver.find_element_by_css_selector(NEXT_PAGE_BUTTON_aliter_1).click()
            except:
                print("Something Weird Happended, Continuing...")
                
        
    FINAL_df = pd.DataFrame()
    for i in range(len(final_matrix)):
        FINAL_df = FINAL_df.append(pd.DataFrame(final_matrix[i],columns = ['Title','Name, Date','Rating','Review']),ignore_index = True)

    hotel_name = driver.find_element_by_css_selector("#detpg_hotel_name").text

    FINAL_df["Hotel Name"] = hotel_name
        
    
    return FINAL_df

In [34]:
min(20,20,20,20)

20

In [ ]:
FINAL = Review_Extractor(driver)

In [271]:
FINAL_df = pd.DataFrame()
for i in range(len(FINAL)):
    FINAL_df = FINAL_df.append(pd.DataFrame(FINAL[i],columns = ['Title','Name, Date','Rating','Review']),ignore_index = True)


In [272]:
FINAL_df

,Title,"Name, Date",Rating,Review
0,Nice service,"Mr. Shivarudrappa, Dec 13, 2019",4.0,Hotel service is very good and the location of...
1,Bad,"Gautham Rao, Dec 05, 2019",1.0,"Terrible experience. Food was pathetic, bathro..."
2,Leisure stay,"Arpana Khatri , Dec 02, 2019",4.0,"The location was very central, nice accommodat..."
3,Nice experience,"DEEPAK SHARMA, Nov 30, 2019",5.0,I booked the room for stay for my father. The ...
4,Safest,"Deepak Ranjan Malla, Nov 30, 2019",4.0,Good clean peaceful and safe place to stay. Af...
...,...,...,...,...
135,"Amazing place, nice hotel and facilities","Anil Sethi, Nov 02, 2018",5.0,Amazing experience. Recommended for everyone
136,Good Stay,"Augustine Maben, Oct 30, 2018",4.0,Pleasant and worthy of money.
137,Excellent Stay,"Sonu Gulfam, Oct 22, 2018",5.0,Everything was good but AC was making a noise.
138,Overall worth,"Indu Bhardwaj, Oct 19, 2018",4.0,I just have to say it's good location.<br>It's...


In [278]:
driver.set_page_load_timeout(15)

'OYO 12075 Hotel Regalia Retreat'

### Putting all the strings together in place now.

In [119]:
def Extract_Reviews_from_links(csv_path,driver):
    links_df = pd.read_csv(csv_path)
    
    try:
        driver = webdriver.Chrome('/home/animesh/Meltwater Winter 2019/Web Scraping Tasks/OYO Rooms Reviews/Booking.com/chromedriver')
    except TimeoutError as te:
        print(te)
        inp = input("Should it continue? on top")
        if((inp == 'y') or (inp == "Y")):
            print("Continuing...")
        else:
            return dataframe
    dataframe = pd.DataFrame()
    for i in range(len(links_df)):
        
        driver.set_page_load_timeout(7)
        try:
            driver.get(links_df.Links.iloc[i+62])
        except :
            print(" Backend Error Encountered...")
            inp = input("Should it continue?")
            if((inp == 'y') or (inp == "Y")):
                print("Continuing...")
                continue
            else:
                return dataframe
            
        time.sleep(1)
        
        print("On the link: {}".format(i+62))
        
        try:
            
            FINAL_df = Review_Extractor(driver)
        except Exception as e:
            print(e)
            print("Some Unexpected Problem has occured, please contact the relevant person.")
            print("                 Meltwater        ")
            time.sleep(2)
            print("             Outside Insight      ")
            time.sleep(1)
            return dataframe
        
        
        
        dataframe = dataframe.append(FINAL_df, ignore_index = True)
        
    return dataframe
    
        
        

In [118]:
driver = webdriver.Chrome(executable_path='/home/animesh/Meltwater Winter 2019/Web Scraping Tasks/OYO Rooms Reviews/Booking.com/chromedriver')

In [120]:
ALL_REVIEWS = Extract_Reviews_from_links('MMT_OYO_Top_Cities_Hotels.csv',driver)

 Backend Error Encountered...


Should it continue? n


In [110]:
ALL_REVIEWS.to_csv("till_56-61.csv")

In [109]:
ALL_REVIEWS

,Title,"Name, Date",Rating,Review,Hotel Name
0,Good rooms in,"Vineesh Vs, Dec 14, 2019",4.0,Is great room and good room service. Staff ind...,OYO 15637 Transit Express
1,Excellent Stay,"Amit Mandal, Dec 07, 2019",5.0,Location is good. The staff were very friendly...,OYO 15637 Transit Express
2,Good,"partha das, Dec 04, 2019",5.0,Excellent will visit soon and recommend,OYO 15637 Transit Express
3,Nice staff,"rakesh sharma, Dec 02, 2019",5.0,"Everything was good, nice room, nice staff.",OYO 15637 Transit Express
4,Oyo transit experience,"Saniyya Mujeeb, Nov 30, 2019",5.0,Well kept and well maintained room and hotel. ...,OYO 15637 Transit Express
...,...,...,...,...,...
439,Good,"Avinash Kumar, Jul 04, 2018",4.0,The location is quote difficult to find but so...,OYO 10921 Stay INN
440,Great hotel,"kunamalla pramod, Jun 09, 2018",5.0,Good location and in a very peaceful neighborh...,OYO 10921 Stay INN
441,Nice stay and spacious rooms,"Gourab Laskar, Jul 31, 2018",5.0,Spacious rooms. Nice stay and good location.,OYO 10921 Stay INN
442,Good,"Avinash Kumar, Jul 04, 2018",4.0,The location is quote difficult to find but so...,OYO 10921 Stay INN


In [57]:
start = time.time()
x= input("wow")
end_time = time.time()-start

wow 1232332werf


In [58]:
end_time

3.532566785812378

Signature: driver.set_page_load_timeout(time_to_wait)
Docstring:
Set the amount of time to wait for a page load to complete
   before throwing an error.

:Args:
 - time_to_wait: The amount of time to wait

:Usage:
    driver.set_page_load_timeout(30)
File:      ~/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py
Type:      method


In [121]:
links_df = pd.read_csv('MMT_OYO_Top_Cities_Hotels.csv')

In [123]:
len(links_df)

62

### Appending them all

In [128]:
files = [file for file in os.listdir('/home/animesh/Meltwater Winter 2019/Web Scraping Tasks/OYO Rooms Reviews/MakeMyTrip.com') if file[:4] == 'till']

In [129]:
files

['till 099 Townhouse.csv',
 'till_Flagship_568_Bandra.csv',
 'till_16545_Hotel_Kant_31.csv',
 'till_14566_Sunshine.csv',
 'till_021_Bombay_Exhib_Centre.csv',
 'till_56-61.csv',
 'till_2105_Hotel_Royal_sheraton_39-44.csv',
 'till_2105_Hotel_Royal_sheraton_45-48.csv',
 'till_49-55.csv']

In [134]:
df = pd.DataFrame()
for f in files:
    wtf = pd.read_csv(f)
    df = df.append(wtf, ignore_index = True)
    print(len(wtf))

340
774
400
674
400
444
541
140
856


In [136]:
df.to_csv("Top Cities MMT OYO Reviews.csv")

In [138]:
df

,Hotel Name,"Name, Date",Rating,Review,Title,Unnamed: 0
0,OYO 12075 Hotel Regalia Retreat,"Mr. Shivarudrappa, Dec 13, 2019",4.0,Hotel service is very good and the location of...,Nice service,0
1,OYO 12075 Hotel Regalia Retreat,"Gautham Rao, Dec 05, 2019",1.0,"Terrible experience. Food was pathetic, bathro...",Bad,1
2,OYO 12075 Hotel Regalia Retreat,"Arpana Khatri , Dec 02, 2019",4.0,"The location was very central, nice accommodat...",Leisure stay,2
3,OYO 12075 Hotel Regalia Retreat,"DEEPAK SHARMA, Nov 30, 2019",5.0,I booked the room for stay for my father. The ...,Nice experience,3
4,OYO 12075 Hotel Regalia Retreat,"Deepak Ranjan Malla, Nov 30, 2019",4.0,Good clean peaceful and safe place to stay. Af...,Safest,4
...,...,...,...,...,...,...
4564,OYO Townhouse 263 The Sky Airport,"Amit Mitra, Jul 15, 2019",4.0,Fair enough. A bit noisy. But overall ok.,Simple,851
4565,OYO Townhouse 263 The Sky Airport,"Konika Anand, Jul 15, 2019",4.0,"Good stay, clean place, comfortable, friendly ...",Good Stay,852
4566,OYO Townhouse 263 The Sky Airport,"Vipin Ravi, Jun 18, 2019",5.0,"A new hotel, close to Airport and with parking...",Very close to Airport with parking space,853
4567,OYO Townhouse 263 The Sky Airport,"Aditya Gangrade, Jun 11, 2019",3.0,They have big and small rooms. They will try t...,Check The rooms before check in,854
